<br>

# Introdução

In [ ]:
#!pip3 install PyPDF2

In [ ]:
import os
import re
import time
import shutil
from zipfile import ZipFile
from PyPDF2 import PdfFileMerger, PdfFileReader

<br>

## Paths

In [ ]:
# Major Path
data_path = os.path.join('..', 'data')

# Paths
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')
output_apartados_path = os.path.join(output_path, 'apartados')

# Clean Directories
shutil.rmtree(output_path, ignore_errors=True)

# Make Directories
os.makedirs(input_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_apartados_path, exist_ok=True)

<br>

# Zip

In [ ]:
def unzip_zipfile(zipfile_file, output_path):
    """
    Para descompactar apenas um arquivo .zip específico
    """
    print('> Etapa 1: Descompacta arquivo')
    with ZipFile(zipfile_file, 'r') as zipObj:
        # Extract all the contents of zip file in different directory
        zipObj.extractall(output_path)

In [ ]:
# ddd
zipfile_file = os.path.join(input_path, '1010642-60.2020.8.26.0019 pequeno.zip')
unzip_zipfile(zipfile_file, output_apartados_path)

<br>

## Função

Função que extrai todo o conteúdo que se encontra entre as palabras "pag " e ").pdf".
Necessária para obter o número da primeira página do documento.

In [ ]:
def extract_text(filename):
    """
    Função que pega o nome do arquivo e cria o nome nome, utilizando
    a primeira página dos documentos extraidos dos documentos do e-SAJ
    imediatamente após o "(pag "...
    """
    # Prefixo e Sufixo da busca, para ser deletado
    start = 'pag '
    end = '.pdf'
    
    # Deleta
    text = re.search('(?<={}).*?(?={})'.format(start, end), filename)
    text = text.group()
    text = text.replace(')', '')
    page_1st = text.split(' - ')[0]
    page_1st = int(page_1st)
    return '{} - {}'.format(page_1st, filename)

In [ ]:
# Test
for i in ['Documento 25 (pag 1249 - 1251).pdf', 'Administrativa (pag 2641).pdf']:
    print(extract_text(i))

<br>

# Principal

In [ ]:
def get_n_files(input_path):
    n_files = 0
    for path, dirs, files in os.walk(input_path):
        n_files += len(files)
    return n_files

In [ ]:
def rename_files(input_files_path):
    print('> Etapa 2: Renomear Arquivos')
    
    # Parameters
    n_file = 0
    n_files = get_n_files(input_files_path)
    
    # Loop
    for path, dirs, files in os.walk(input_files_path):
        for file in files:
            n_file+=1
            input_file = os.path.join(path, file)
            output_filename = extract_text(file)
            output_file = os.path.join(input_files_path, output_filename)
            print('Arquivo {} de {} renomeado - {}% concluído. Aguarde.'.format(n_file, n_files, int(n_file/n_files*100)), end='\r')
            os.rename(input_file, output_file)
    
    # 
    print("{:<100}".format('> Etapa 2: Concluída.'), end='\r')

In [ ]:
# Renomeia os arquivos
rename_files(output_apartados_path)

<br>

# Merge PDFs

In [ ]:
def adjust_bookmark(filename):
    """
    Cria um nome para o bookmark a partir do nome do arquivo.
    """
    bookmark = filename.split(' - ', maxsplit=1)[-1]
    bookmark = bookmark.replace('.pdf', '')
    return bookmark

In [ ]:
# Test
adjust_bookmark('208 - Documento 4 (pag 208 - 209).pdf')

In [ ]:
def get_int(name):
    """
    Custom Function to sort list of files, based in their names.
    """
    num = name.split(' - ')[0]
    return int(num)


def sort_files_as_list(path):
    """
    Sort list os files in directort
    """
    # List Files
    list_files = os.listdir(path)

    # Sort
    list_files.sort(key=get_int)
    return list_files

In [ ]:
# Sort Files
list_files = sort_files_as_list(output_apartados_path)

In [ ]:
def merge_files(input_files_path, output_file_path, filename):
    # Append PDF files sorted
    print('> Etapa 3: Unifica Arquivos')

    # List Files
    list_files = sort_files_as_list(input_files_path)

    # Parameters
    n_file = 0
    n_files = get_n_files(input_files_path)

    # Call the PdfFileMerger
    merged_object = PdfFileMerger()
    
    # Loop
    for file in list_files:
        n_file+=1
        bookmark = adjust_bookmark(file)
        merged_object.append(
            PdfFileReader(os.path.join(input_files_path, file), 'rb'),
            bookmark,
        )
        print('Arquivo {} de {} juntado - {}% concluído. Aguarde.'.format(n_file, n_files, int(n_file/n_files*100)), end = '\r')

    # Write all the files into a file which is named as shown below
    merged_object.write(os.path.join(output_file_path, filename))
    
    # Fim
    print("{:<100}".format('> Etapa 3: Concluída.'), end='\r')
    return 0

In [ ]:
def create_output_filename(zipfile_file):
    output_filename = os.path.basename(zipfile_file)
    output_filename = output_filename.replace('.zip', '.pdf')
    return output_filename

In [ ]:
# Output Filename
output_filename = create_output_filename(zipfile_file)
output_filename

In [ ]:
# Merge Files
merge_files(output_apartados_path, output_path, output_filename)

# Clean Directories
shutil.rmtree(output_apartados_path, ignore_errors=True)

<br>

# Export

In [ ]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [ ]:
filename = 'fix_filenames.ipynb'
notebook = os.path.join(os.getcwd(), filename)

In [ ]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(notebook)

# Write to output html file
with open(os.path.join('..', 'src', 'fix_filenames.py'),  'w', encoding='utf-8') as f:
    f.write(body)